# Introduction to Flask
In this demo we will take show how to serve models with Flask as part of Model deployment.

In [ ]:
# Install Flask
!pip install Flask

In [ ]:
# RESTART THE NOTEBOOK

In [ ]:
# Show the version
!python -m flask --version

In [ ]:
# Setup Structure for Flask App
!tree flask_app/

# Create a Flask App
We will follow along in the cells alongside our `app.py` file.

## Loading a model in Flask
```py
# Initialize Flask app
app = Flask(__name__)

# Load any other environment variables or variables here
MY_SECRET = os.getenv('SECRET')

# Load the MobileNetV3 Large pre-trained model
model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.DEFAULT)
model.eval()  # Set model to evaluation mode

```

## Create an endpoint
```py
# Map URL to an endpoint and method
@app.route('/predict', methods=['POST'])
def predict():
    # Retrieve the request
    data = request.json
    
    # Transform input if needed
    input_tensor = torch.tensor(data['input'])

    # Get the prediction
    output = model(input_tensor)
    
    # Return a response
    return jsonify({'output': output.tolist()})

```

## request
The `request` module in this code is part of Flask and is used to access data from incoming HTTP requests. 

Here, `request.json` parses the JSON payload sent with the POST request and converts it into a Python dictionary for easy access and processing.

```py
from flask import request

@app.route('/predict', methods=['POST'])
def predict():
    # Retrieve the request
    data = request.json
```

## jsonify
`jsonify` is a Flask utility that converts Python dictionaries, lists, or other serializable objects into JSON format, which is the standard data format for API responses. 

It also sets the correct Content-Type header (application/json) in the HTTP response, ensuring clients recognize the data as JSON.

```py
from flask import jsonify

# Return a response
return jsonify({'output': output.tolist()})

```


## Health Endpoint
If you can get a response, the server is healthy and running.

```py
# Health endpoint
@app.route('/health', methods=['GET'])
def health():
    """
    Health check endpoint to confirm the app is running.
    """
    return jsonify({'status': 'healthy'}), 200
```

## Logging
Logging in the app to see what is happening and see if/where errors occur.

```py
except Exception as e:
        logger.error(f"Error during prediction: {str(e)}")
        response = {'error': str(e)}
        logger.info(f"Response for /predict: {response}")
        
        return jsonify(response), 500
```

## Error Handling
A best practice for a Production deployment.

```py
# Prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    # Error handling
    try:
        # Extract Base64 string from request JSON
        data = request.json
        if 'image' not in data:
            # Return and error if there is no image in the request
            return jsonify({'error': 'No image provided'}), 400
        
        # Decode the Base64 image string
        image_data = base64.b64decode(data['image'])
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
        
        # Preprocess the image
        transformed_img = preprocess(image).unsqueeze(0)
        
        # Perform inference
        with torch.no_grad():
            output = model(transformed_img)
            _, predicted = torch.max(output.data, 1)
            print(predicted)
        
        # Return our prediction
        return jsonify({'prediction': predicted.item()})
    
    # Fail with our error in the response
    except Exception as e:
        return jsonify({'error': str(e)}), 500
```

## Run Flask server (Development mode)

```bash
> python app.py
```

## Send a request
Sending binary data over HTTP.

In [ ]:
import base64

with open('dog-1.jpg', 'rb') as img_file:
    base64_string = base64.b64encode(img_file.read()).decode('utf-8')
    print(base64_string)


In [32]:
# Test a request using python requests library
import requests

# JSON payload with the Base64 encoded image
payload = {
    "image": base64_string
}

# Set the headers
headers = {
    "Content-Type": "application/json"
}

In [ ]:
# Send POST request
response = requests.post("http://127.0.0.1:5000/predict", 
                         json=payload, 
                         headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

In [ ]:
# Check the health endpoint
response = requests.get("http://127.0.0.1:5000/health")

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

## Test Error Handling
Be sure to also check the logs.

In [ ]:
# Test error handling
response = requests.post("http://127.0.0.1:5000/predict",
                         headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

In [ ]:
# Send POST request
response = requests.post("http://127.0.0.1:5000/predict", 
                         json={"video": base64_string}, 
                         headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

## Running Flask Server with Gunicorn
This is considered a better way to serve a Flask app.

## Using Gunicorn
```bash
> gunicorn -w 2 -b 0.0.0.0:8080 app:app
```

In [ ]:
## Send Request
# Send POST request
response = requests.post("http://127.0.0.1:8080/predict", 
                         json=payload, 
                         headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

In [ ]:
import json 

# Downloaded from Hugging Face
# https://huggingface.co/datasets/huggingface/label-files/blob/main/imagenet-1k-id2label.json
with open("labels.json", "r") as f:
    imagenet_classes = json.load(f)


In [ ]:
# Get the actual class name from our labels
imagenet_classes['207']